In [16]:
#prepare labels
file = open('mscoco_label_map.pbtxt.txt')
labels = ["null"] #because the first label with an id 1 in the label_map.pbtxt
txt =file.read()
line = txt.split('\nitem')
ind= [i.split('display_name: "') for i in line]
for i in range(len(ind)):
    labels.append(ind[i][1].replace('"\n}',''))
labels[:10]

['null',
 'person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat']

In [27]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
model_path = 'ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb'
config_file = 'ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt'
model = cv2.dnn.readNetFromTensorflow(model_path,config_file)
filename = 'outpy.avi'
cap =cv2.VideoCapture(filename)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
file_size = (frame_width,frame_height)
# We want to save the output to a video file
output_filename = 'detected_objects.avi'
output_frames_per_second = 20

#Create a VideoWriter object so we can save the video output
writer = cv2.VideoWriter_fourcc('M','J','P','G')
result = cv2.VideoWriter(output_filename,writer, output_frames_per_second,file_size ) 
while True:
    success, img = cap.read()
    if success:
        base_img = img.copy()
        original_size = img.shape
        tartget_size = (300,300)
        img = cv2.resize(img,tartget_size)
        aspect_ratio_y = original_size[0] / tartget_size [0] #help us in to apply the work on base_img
        aspect_ratio_x = original_size[1] / tartget_size[1]
        imgblob= cv2.dnn.blobFromImage(img) #prepare img and returen 4-dim (1,3,300,300)
        model.setInput(imgblob)
        detections = model.forward()
        detections_df = pd.DataFrame(detections[0][0], columns = ["img_id", "label", "confidence", "left", "top", "right", "bottom"])
        detections_df = detections_df.query('confidence > 0.9') #label of 1 means face\ 0:means background
        for i,instance in detections_df.iterrows():
            text = labels[int(detections_df['label'][i])] + str(round(detections_df['confidence'][i]*100)) + "%"
            right = int(instance['right'] * 300) 
            left = int(instance['left'] * 300)
            top = int(instance['top'] * 300)
            bottom = int(instance ['bottom'] * 300)
            detect_face = base_img[int(top *aspect_ratio_y): int(bottom*aspect_ratio_y), int(left*aspect_ratio_x): int(right*aspect_ratio_x)]
            if detect_face.shape[0] > 0 and detect_face.shape[1] > 0:
                cv2.putText(base_img, text , (int(left*aspect_ratio_x), int(top*aspect_ratio_y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 200), 2)
                cv2.rectangle(base_img, (int(left*aspect_ratio_x), int(top*aspect_ratio_y)), (int(right*aspect_ratio_x), int(bottom*aspect_ratio_y)), (255, 0, 0), 4) #draw rectangle to main ima
        
        result.write(base_img)
    else:
        break

        
    
    
cap.release()    
cv2.destroyAllWindows()
